### This is a script to run periodic model performance evaluation using the most recent 1 day of data

In [83]:
#### first get 2mon data and 1d dta. then join them. then reuse predict script.

In [96]:
import numpy as np
import pandas as pd
import os, time, warnings, random, shap, requests, optuna, datetime, joblib, pytz
import seaborn as sns
import matplotlib.pyplot as plt
import functools as ft
import yfinance as yf

from sklearn.compose import ColumnTransformer, make_column_transformer
from sklearn.preprocessing import LabelBinarizer, LabelEncoder, OrdinalEncoder, OneHotEncoder, StandardScaler
from sklearn.linear_model import LinearRegression, Lasso, Ridge, ElasticNet
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV, train_test_split, KFold
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier, VotingClassifier
from sklearn.metrics import accuracy_score, f1_score, r2_score, mean_squared_error
from sklearn.inspection import permutation_importance
from xgboost import XGBClassifier, XGBRegressor

pd.set_option('display.max_columns', 100)
pd.set_option('mode.chained_assignment', None)
pd.set_option('display.expand_frame_repr', False)
warnings.filterwarnings('ignore') 

time0 = time.time()

os.chdir('/home/jupyter/projects_gcp_cpu/spx/src')
os.getcwd()

'/home/jupyter/projects_gcp_cpu/spx/src'

In [85]:
df = pd.read_csv('gs://pmykola-streaming-projects/spg-stocks/data/data_start_20221021.csv')
df.Datetime = pd.to_datetime(df.Datetime)
df

,Datetime,EEM,EEMA,EMXC,Nasdaq,Russel,Spx,VTHR
0,2022-10-21 09:30:00,34.634998,55.570000,45.400002,10576.550781,1710.077637,3659.689941,164.059998
1,2022-10-21 09:32:00,34.654999,55.660000,45.410000,10586.665039,1709.384399,3660.110107,164.169998
2,2022-10-21 09:34:00,34.610001,55.619999,45.360001,10547.528320,1704.969971,3652.659912,164.020004
3,2022-10-21 09:36:00,34.709999,NaN,45.498699,10586.677734,1705.815674,3665.989990,NaN
4,2022-10-21 09:38:00,34.689999,NaN,45.463501,10583.666016,1704.521973,3662.169922,163.869995
...,...,...,...,...,...,...,...,...
5952,2022-12-05 15:52:00,39.084999,NaN,49.470001,11222.517578,1837.032227,3995.699951,NaN
5953,2022-12-05 15:54:00,39.084999,NaN,49.470001,11231.310547,1838.013672,3998.479980,179.253998
5954,2022-12-05 15:56:00,39.105000,64.910004,49.480000,11241.674805,1839.835571,4000.719971,179.289993
5955,2022-12-05 15:58:00,39.084999,65.019997,49.490002,11242.072266,1840.386230,3999.149902,179.500000


In [86]:
df1d = pd.read_csv('gs://pmykola-streaming-projects/spg-stocks/data/data_last_2022126.csv')
df1d.Datetime = pd.to_datetime(df1d.Datetime)
df1d
# this is not what i see when i open this file. wrong date pasring?

,Datetime,EEM,EEMA,EMXC,Nasdaq,Russel,Spx,VTHR
0,2022-12-06 09:30:00,39.240002,65.089996,49.399899,11229.620117,1836.926636,3999.110107,179.300003
1,2022-12-06 09:32:00,39.230000,65.110001,49.389999,11220.044922,1837.946777,4000.229980,179.360001
2,2022-12-06 09:34:00,39.189999,65.040001,49.389999,11208.439453,1837.892456,3996.840088,NaN
3,2022-12-06 09:36:00,39.230000,65.019997,49.419998,11215.238281,1838.812744,3998.750000,179.259995
4,2022-12-06 09:38:00,39.200001,65.029999,49.419998,11206.872070,1838.209717,3995.010010,179.220001
...,...,...,...,...,...,...,...,...
191,2022-12-06 15:52:00,39.064999,NaN,49.200001,11018.576172,1811.978882,3941.929932,NaN
192,2022-12-06 15:54:00,39.070000,NaN,49.200001,11017.687500,1811.951904,3941.500000,NaN
193,2022-12-06 15:56:00,39.064999,NaN,49.180000,11008.847656,NaN,3938.770020,NaN
194,2022-12-06 15:58:00,39.040001,NaN,49.130001,11015.938477,NaN,3941.500000,NaN


In [87]:
df = pd.concat([df, df1d])
df.sort_values(by='Datetime')
df

,Datetime,EEM,EEMA,EMXC,Nasdaq,Russel,Spx,VTHR
0,2022-10-21 09:30:00,34.634998,55.570000,45.400002,10576.550781,1710.077637,3659.689941,164.059998
1,2022-10-21 09:32:00,34.654999,55.660000,45.410000,10586.665039,1709.384399,3660.110107,164.169998
2,2022-10-21 09:34:00,34.610001,55.619999,45.360001,10547.528320,1704.969971,3652.659912,164.020004
3,2022-10-21 09:36:00,34.709999,NaN,45.498699,10586.677734,1705.815674,3665.989990,NaN
4,2022-10-21 09:38:00,34.689999,NaN,45.463501,10583.666016,1704.521973,3662.169922,163.869995
...,...,...,...,...,...,...,...,...
191,2022-12-06 15:52:00,39.064999,NaN,49.200001,11018.576172,1811.978882,3941.929932,NaN
192,2022-12-06 15:54:00,39.070000,NaN,49.200001,11017.687500,1811.951904,3941.500000,NaN
193,2022-12-06 15:56:00,39.064999,NaN,49.180000,11008.847656,NaN,3938.770020,NaN
194,2022-12-06 15:58:00,39.040001,NaN,49.130001,11015.938477,NaN,3941.500000,NaN


In [88]:

df['time'] = df.Datetime.dt.time
df['date'] = df.Datetime.dt.date

df = df.fillna(method='ffill')
dayclose = df[df.time==datetime.time(15, 58, 0)]
dayopen = df[df.time==datetime.time(9, 30, 0)]
dayopen.reset_index(drop=True, inplace=True)
dayclose.reset_index(drop=True, inplace=True)
dayclose.sort_values(by='date')
display(df)

df

,Datetime,EEM,EEMA,EMXC,Nasdaq,Russel,Spx,VTHR,time,date
0,2022-10-21 09:30:00,34.634998,55.570000,45.400002,10576.550781,1710.077637,3659.689941,164.059998,09:30:00,2022-10-21
1,2022-10-21 09:32:00,34.654999,55.660000,45.410000,10586.665039,1709.384399,3660.110107,164.169998,09:32:00,2022-10-21
2,2022-10-21 09:34:00,34.610001,55.619999,45.360001,10547.528320,1704.969971,3652.659912,164.020004,09:34:00,2022-10-21
3,2022-10-21 09:36:00,34.709999,55.619999,45.498699,10586.677734,1705.815674,3665.989990,164.020004,09:36:00,2022-10-21
4,2022-10-21 09:38:00,34.689999,55.619999,45.463501,10583.666016,1704.521973,3662.169922,163.869995,09:38:00,2022-10-21
...,...,...,...,...,...,...,...,...,...,...
191,2022-12-06 15:52:00,39.064999,64.720001,49.200001,11018.576172,1811.978882,3941.929932,176.699997,15:52:00,2022-12-06
192,2022-12-06 15:54:00,39.070000,64.720001,49.200001,11017.687500,1811.951904,3941.500000,176.699997,15:54:00,2022-12-06
193,2022-12-06 15:56:00,39.064999,64.720001,49.180000,11008.847656,1811.951904,3938.770020,176.699997,15:56:00,2022-12-06
194,2022-12-06 15:58:00,39.040001,64.720001,49.130001,11015.938477,1811.951904,3941.500000,176.699997,15:58:00,2022-12-06


,Datetime,EEM,EEMA,EMXC,Nasdaq,Russel,Spx,VTHR,time,date
0,2022-10-21 09:30:00,34.634998,55.570000,45.400002,10576.550781,1710.077637,3659.689941,164.059998,09:30:00,2022-10-21
1,2022-10-21 09:32:00,34.654999,55.660000,45.410000,10586.665039,1709.384399,3660.110107,164.169998,09:32:00,2022-10-21
2,2022-10-21 09:34:00,34.610001,55.619999,45.360001,10547.528320,1704.969971,3652.659912,164.020004,09:34:00,2022-10-21
3,2022-10-21 09:36:00,34.709999,55.619999,45.498699,10586.677734,1705.815674,3665.989990,164.020004,09:36:00,2022-10-21
4,2022-10-21 09:38:00,34.689999,55.619999,45.463501,10583.666016,1704.521973,3662.169922,163.869995,09:38:00,2022-10-21
...,...,...,...,...,...,...,...,...,...,...
191,2022-12-06 15:52:00,39.064999,64.720001,49.200001,11018.576172,1811.978882,3941.929932,176.699997,15:52:00,2022-12-06
192,2022-12-06 15:54:00,39.070000,64.720001,49.200001,11017.687500,1811.951904,3941.500000,176.699997,15:54:00,2022-12-06
193,2022-12-06 15:56:00,39.064999,64.720001,49.180000,11008.847656,1811.951904,3938.770020,176.699997,15:56:00,2022-12-06
194,2022-12-06 15:58:00,39.040001,64.720001,49.130001,11015.938477,1811.951904,3941.500000,176.699997,15:58:00,2022-12-06


In [89]:
df

,Datetime,EEM,EEMA,EMXC,Nasdaq,Russel,Spx,VTHR,time,date
0,2022-10-21 09:30:00,34.634998,55.570000,45.400002,10576.550781,1710.077637,3659.689941,164.059998,09:30:00,2022-10-21
1,2022-10-21 09:32:00,34.654999,55.660000,45.410000,10586.665039,1709.384399,3660.110107,164.169998,09:32:00,2022-10-21
2,2022-10-21 09:34:00,34.610001,55.619999,45.360001,10547.528320,1704.969971,3652.659912,164.020004,09:34:00,2022-10-21
3,2022-10-21 09:36:00,34.709999,55.619999,45.498699,10586.677734,1705.815674,3665.989990,164.020004,09:36:00,2022-10-21
4,2022-10-21 09:38:00,34.689999,55.619999,45.463501,10583.666016,1704.521973,3662.169922,163.869995,09:38:00,2022-10-21
...,...,...,...,...,...,...,...,...,...,...
191,2022-12-06 15:52:00,39.064999,64.720001,49.200001,11018.576172,1811.978882,3941.929932,176.699997,15:52:00,2022-12-06
192,2022-12-06 15:54:00,39.070000,64.720001,49.200001,11017.687500,1811.951904,3941.500000,176.699997,15:54:00,2022-12-06
193,2022-12-06 15:56:00,39.064999,64.720001,49.180000,11008.847656,1811.951904,3938.770020,176.699997,15:56:00,2022-12-06
194,2022-12-06 15:58:00,39.040001,64.720001,49.130001,11015.938477,1811.951904,3941.500000,176.699997,15:58:00,2022-12-06


In [90]:
# get code from prediction, then modify it

In [91]:
dayopen

,Datetime,EEM,EEMA,EMXC,Nasdaq,Russel,Spx,VTHR,time,date
0,2022-10-21 09:30:00,34.634998,55.570000,45.400002,10576.550781,1710.077637,3659.689941,164.059998,09:30:00,2022-10-21
1,2022-10-24 09:30:00,33.950001,54.200001,45.580002,10831.625000,1744.627563,3765.520020,168.860001,09:30:00,2022-10-24
2,2022-10-25 09:30:00,34.084999,54.450001,45.431999,11022.798828,1753.145142,3805.649902,170.169998,09:30:00,2022-10-25
3,2022-10-26 09:30:00,34.349998,54.880001,45.830002,10956.992188,1803.124023,3829.889893,171.710007,09:30:00,2022-10-26
4,2022-10-27 09:30:00,34.590000,55.560001,46.139999,10987.374023,1822.050537,3847.090088,173.000000,09:30:00,2022-10-27
5,2022-10-28 09:30:00,34.029999,54.349998,45.889999,10801.363281,1811.265869,3818.780029,171.449997,09:30:00,2022-10-28
6,2022-10-31 09:30:00,34.005001,54.330002,45.919998,11043.416016,1838.572754,3878.120117,174.235001,09:30:00,2022-10-31
7,2022-11-01 09:30:00,35.070000,56.119999,46.930000,11133.355469,1866.141724,3904.679932,175.869995,09:30:00,2022-11-01
8,2022-11-02 09:30:00,34.930000,56.090000,46.665001,10865.519531,1839.983398,3842.489990,173.199997,09:30:00,2022-11-02
9,2022-11-03 09:30:00,34.325001,54.930000,45.880001,10401.338867,1765.190186,3716.340088,166.960007,09:30:00,2022-11-03


In [92]:
### now i wanna do feature engineering for all assets 

asset_list = ['Spx', 'Nasdaq', 'Russel', 'EMXC', 'EEMA', 'EEM', 'VTHR']

for asset in asset_list:
    
    df[asset + '_ret'] = 100*(df[asset]/df[asset].shift(1)-1)
    df['s_' + asset + '_ret_1prd'] = (100*(df[asset]/df[asset].shift(1)-1)).shift(1)
    df['s_' + asset + '_ret_2prd'] = (100*(df[asset]/df[asset].shift(2)-1)).shift(1)
    df['s_' + asset + '_ret_4prd'] = (100*(df[asset]/df[asset].shift(4)-1)).shift(1)
    # display(df.shape, df.head(5))

    df.loc[df.time < datetime.time(9, 32, 0), 's_' + asset + '_1prd'] = np.nan
    df.loc[df.time < datetime.time(9, 33, 0), 's_' + asset + '_2prd'] = np.nan
    df.loc[df.time < datetime.time(9, 35, 0), 's_' + asset + '_4prd'] = np.nan

    dayopen.rename(columns={asset:asset+'_open'}, inplace=True)
    # dayopen.head()
    dayclose.rename(columns={asset:asset+'_close'}, inplace=True)
    dayclose_l1 = dayclose.copy()
    dayclose_l2 = dayclose.copy()
    dayclose_l1[asset+'_close_l1'] = dayclose_l1[asset+'_close'].shift(1)
    dayclose_l2[asset+'_close_l2'] = dayclose_l2[asset+'_close'].shift(2)

    # display(dayclose_l1.head(), dayclose_l2.head())

    df = pd.merge(df, dayopen[['date', asset + '_open']], on=['date'], how='left')
    df = pd.merge(df, dayclose_l1[['date', asset + '_close_l1']], on=['date'], how='left')
    df = pd.merge(df, dayclose_l2[['date', asset + '_close_l2']], on=['date'], how='left')

    df['s_' + asset + '_ret_open'] = (100*(df[asset]/df[asset + '_open']-1)).shift(1)
    df['s_' + asset + '_ret_close1'] = (100*(df[asset]/df[asset + '_close_l1']-1)).shift(1)
    df['s_' + asset + '_ret_close2'] = (100*(df[asset]/df[asset + '_close_l2']-1)).shift(1)

    cols_todrop = [x for x in list(df.columns) if asset in x and 'ret' not in x]
    df.drop(columns = cols_todrop, inplace=True)

display(time.time() - time0, df.tail())

17.68523645401001

,Datetime,time,date,Spx_ret,s_Spx_ret_1prd,s_Spx_ret_2prd,s_Spx_ret_4prd,s_Spx_ret_open,s_Spx_ret_close1,s_Spx_ret_close2,Nasdaq_ret,s_Nasdaq_ret_1prd,s_Nasdaq_ret_2prd,s_Nasdaq_ret_4prd,s_Nasdaq_ret_open,s_Nasdaq_ret_close1,s_Nasdaq_ret_close2,Russel_ret,s_Russel_ret_1prd,s_Russel_ret_2prd,s_Russel_ret_4prd,s_Russel_ret_open,s_Russel_ret_close1,s_Russel_ret_close2,EMXC_ret,s_EMXC_ret_1prd,s_EMXC_ret_2prd,s_EMXC_ret_4prd,s_EMXC_ret_open,s_EMXC_ret_close1,s_EMXC_ret_close2,EEMA_ret,s_EEMA_ret_1prd,s_EEMA_ret_2prd,s_EEMA_ret_4prd,s_EEMA_ret_open,s_EEMA_ret_close1,s_EEMA_ret_close2,EEM_ret,s_EEM_ret_1prd,s_EEM_ret_2prd,s_EEM_ret_4prd,s_EEM_ret_open,s_EEM_ret_close1,s_EEM_ret_close2,VTHR_ret,s_VTHR_ret_1prd,s_VTHR_ret_2prd,s_VTHR_ret_4prd,s_VTHR_ret_open,s_VTHR_ret_close1,s_VTHR_ret_close2
6148,2022-12-06 15:52:00,15:52:00,2022-12-06,0.001016,0.006343,0.083273,0.108944,-1.430824,-1.431805,-3.184540,-0.014232,-0.005742,0.067544,0.132515,-1.865384,-1.974082,-3.852346,0.025202,0.055307,0.072364,0.155268,-1.382978,-1.568361,-4.290911,0.005490,0.025005,0.055525,0.116607,-0.410119,-0.591436,-2.347561,0.0,0.0,0.0,0.030919,-0.568436,-0.461389,-1.190838,0.000000,-0.012546,0.064031,0.102491,-0.445981,-0.051172,-1.226297,0.0,0.118982,0.118982,0.392025,-1.450087,-1.55989,-3.363418
6149,2022-12-06 15:54:00,15:54:00,2022-12-06,-0.010907,0.001016,0.007358,0.064478,-1.429822,-1.430803,-3.183557,-0.008065,-0.014232,-0.019973,0.051928,-1.879351,-1.988033,-3.866030,-0.001489,0.025202,0.080523,0.133291,-1.358125,-1.543554,-4.266790,0.000000,0.005490,0.030496,0.077090,-0.404652,-0.585979,-2.342200,0.0,0.0,0.0,0.000000,-0.568436,-0.461389,-1.190838,0.012802,0.000000,-0.012546,0.064031,-0.445981,-0.051172,-1.226297,0.0,0.000000,0.118982,0.118982,-1.450087,-1.55989,-3.363418
6150,2022-12-06 15:56:00,15:56:00,2022-12-06,-0.069262,-0.010907,-0.009891,0.073373,-1.440573,-1.441554,-3.194116,-0.080233,-0.008065,-0.022296,0.045234,-1.887264,-1.995938,-3.873783,0.000000,-0.001489,0.023713,0.096094,-1.359593,-1.545020,-4.268215,-0.040651,0.000000,0.005490,0.061018,-0.404652,-0.585979,-2.342200,0.0,0.0,0.0,0.000000,-0.568436,-0.461389,-1.190838,-0.012800,0.012802,0.012802,0.076841,-0.433236,-0.038376,-1.213652,0.0,0.000000,0.000000,0.118982,-1.450087,-1.55989,-3.363418
6151,2022-12-06 15:58:00,15:58:00,2022-12-06,0.069310,-0.069262,-0.080162,-0.072809,-1.508838,-1.509818,-3.261167,0.064410,-0.080233,-0.088292,-0.108247,-1.965983,-2.074570,-3.950908,0.000000,0.000000,-0.001489,0.079033,-1.359593,-1.545020,-4.268215,-0.101666,-0.040651,-0.040651,-0.010168,-0.445139,-0.626392,-2.381899,0.0,0.0,0.0,0.000000,-0.568436,-0.461389,-1.190838,-0.063990,-0.012800,0.000000,-0.012546,-0.445981,-0.051172,-1.226297,0.0,0.000000,0.000000,0.118982,-1.450087,-1.55989,-3.363418
6152,2022-12-06 16:00:00,16:00:00,2022-12-06,-0.006089,0.069310,0.000000,-0.009891,-1.440573,-1.441554,-3.194116,-0.009521,0.064410,-0.015875,-0.038167,-1.902839,-2.011496,-3.889043,0.000000,0.000000,0.000000,0.023713,-1.359593,-1.545020,-4.268215,0.000000,-0.101666,-0.142276,-0.136794,-0.546352,-0.727421,-2.481143,0.0,0.0,0.0,0.000000,-0.568436,-0.461389,-1.190838,0.000000,-0.063990,-0.076782,-0.063990,-0.509686,-0.115129,-1.289503,0.0,0.000000,0.000000,0.000000,-1.450087,-1.55989,-3.363418


In [111]:
### do prediction ###

model_path = '/home/jupyter/project_repos/spg_stocks/spg_stocks/stocks-app/en_model.pkl'
trained_model = joblib.load(open(model_path, "rb"))

X = df.loc[df.date == df.date.max()]
X.drop(columns = ['Datetime',
                  'time', 
                  'date', 
                  'Spx_ret', 
                  'Nasdaq_ret', 
                  'Russel_ret', 
                  'EEMA_ret', 
                  'EEM_ret', 
                  'EMXC_ret', 
                  'VXUS_ret', 
                  'VTHR_ret'], 
                  inplace=True,
                  errors = 'ignore')

if(X.count().sum() < X.shape[1]):
    print(f'''There are {X.shape[1] - X.count().sum()} missing values. 
          There will be an error''')

print(trained_model.predict(X))

print('Total time: ', time.time()-time0)

[-1.00270850e-02  3.10244864e-03 -1.59283868e-03 -9.56831996e-03
  1.04613715e-02  3.00963112e-03 -9.28997201e-03  1.16135734e-02
 -2.44664102e-02 -1.76634144e-02  2.05536235e-02  1.75183287e-02
  1.61519359e-02 -6.47644132e-03 -2.05835161e-02 -3.31805122e-02
 -3.08220270e-02 -4.94383657e-02 -5.97910913e-02 -8.60125236e-02
 -6.52782833e-02 -5.86651758e-02 -6.61542756e-02  5.57115306e-03
  1.20939166e-02  1.90413243e-02  2.08477634e-02  1.87055371e-02
  2.61045494e-02  2.01436702e-02 -6.70673001e-03 -3.97684330e-03
  1.91780091e-03 -1.47132028e-02 -1.31999514e-02 -3.19584375e-02
 -5.08638338e-02 -2.83574276e-02  2.11177178e-03 -1.42627900e-03
 -4.85468314e-03 -3.71318084e-03 -7.35028701e-03 -1.74932222e-02
 -2.67112764e-02 -2.92305101e-02 -2.01711283e-02  1.69674718e-02
  2.41429620e-02  2.91523414e-02  1.10970372e-02  3.82033532e-02
  1.95174960e-02 -1.06880918e-02 -8.08623721e-03  3.60165477e-03
  1.68529343e-02  2.84397763e-02  2.42899439e-02  2.33779472e-02
  2.27821317e-02  3.46129

In [ ]:
# evaluate performance

In [112]:
df

,Datetime,time,date,Spx_ret,s_Spx_ret_1prd,s_Spx_ret_2prd,s_Spx_ret_4prd,s_Spx_ret_open,s_Spx_ret_close1,s_Spx_ret_close2,Nasdaq_ret,s_Nasdaq_ret_1prd,s_Nasdaq_ret_2prd,s_Nasdaq_ret_4prd,s_Nasdaq_ret_open,s_Nasdaq_ret_close1,s_Nasdaq_ret_close2,Russel_ret,s_Russel_ret_1prd,s_Russel_ret_2prd,s_Russel_ret_4prd,s_Russel_ret_open,s_Russel_ret_close1,s_Russel_ret_close2,EMXC_ret,s_EMXC_ret_1prd,s_EMXC_ret_2prd,s_EMXC_ret_4prd,s_EMXC_ret_open,s_EMXC_ret_close1,s_EMXC_ret_close2,EEMA_ret,s_EEMA_ret_1prd,s_EEMA_ret_2prd,s_EEMA_ret_4prd,s_EEMA_ret_open,s_EEMA_ret_close1,s_EEMA_ret_close2,EEM_ret,s_EEM_ret_1prd,s_EEM_ret_2prd,s_EEM_ret_4prd,s_EEM_ret_open,s_EEM_ret_close1,s_EEM_ret_close2,VTHR_ret,s_VTHR_ret_1prd,s_VTHR_ret_2prd,s_VTHR_ret_4prd,s_VTHR_ret_open,s_VTHR_ret_close1,s_VTHR_ret_close2
0,2022-10-21 09:30:00,09:30:00,2022-10-21,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2022-10-21 09:32:00,09:32:00,2022-10-21,0.011481,NaN,NaN,NaN,0.000000,NaN,NaN,0.095629,NaN,NaN,NaN,0.000000,NaN,NaN,-0.040538,NaN,NaN,NaN,0.000000,NaN,NaN,0.022023,NaN,NaN,NaN,0.000000,NaN,NaN,0.161958,NaN,NaN,NaN,0.000000,NaN,NaN,0.057746,NaN,NaN,NaN,0.000000,NaN,NaN,0.067049,NaN,NaN,NaN,0.000000,NaN,NaN
2,2022-10-21 09:34:00,09:34:00,2022-10-21,-0.203551,0.011481,NaN,NaN,0.011481,NaN,NaN,-0.369679,0.095629,NaN,NaN,0.095629,NaN,NaN,-0.258247,-0.040538,NaN,NaN,-0.040538,NaN,NaN,-0.110106,0.022023,NaN,NaN,0.022023,NaN,NaN,-0.071867,0.161958,NaN,NaN,0.161958,NaN,NaN,-0.129846,0.057746,NaN,NaN,0.057746,NaN,NaN,-0.091365,0.067049,NaN,NaN,0.067049,NaN,NaN
3,2022-10-21 09:36:00,09:36:00,2022-10-21,0.364942,-0.203551,-0.192094,NaN,-0.192094,NaN,NaN,0.371171,-0.369679,-0.274404,NaN,-0.274404,NaN,NaN,0.049602,-0.258247,-0.298680,NaN,-0.298680,NaN,NaN,0.305773,-0.110106,-0.088108,NaN,-0.088108,NaN,NaN,0.000000,-0.071867,0.089975,NaN,0.089975,NaN,NaN,0.288929,-0.129846,-0.072175,NaN,-0.072175,NaN,NaN,0.000000,-0.091365,-0.024377,NaN,-0.024377,NaN,NaN
4,2022-10-21 09:38:00,09:38:00,2022-10-21,-0.104203,0.364942,0.160648,NaN,0.172147,NaN,NaN,-0.028448,0.371171,0.000120,NaN,0.095749,NaN,NaN,-0.075841,0.049602,-0.208773,NaN,-0.249226,NaN,NaN,-0.077361,0.305773,0.195330,NaN,0.217396,NaN,NaN,0.000000,0.000000,-0.071867,NaN,0.089975,NaN,NaN,-0.057622,0.288929,0.158708,NaN,0.216546,NaN,NaN,-0.091458,0.000000,-0.091365,NaN,-0.024377,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6148,2022-12-06 15:52:00,15:52:00,2022-12-06,0.001016,0.006343,0.083273,0.108944,-1.430824,-1.431805,-3.184540,-0.014232,-0.005742,0.067544,0.132515,-1.865384,-1.974082,-3.852346,0.025202,0.055307,0.072364,0.155268,-1.382978,-1.568361,-4.290911,0.005490,0.025005,0.055525,0.116607,-0.410119,-0.591436,-2.347561,0.000000,0.000000,0.000000,0.030919,-0.568436,-0.461389,-1.190838,0.000000,-0.012546,0.064031,0.102491,-0.445981,-0.051172,-1.226297,0.000000,0.118982,0.118982,0.392025,-1.450087,-1.55989,-3.363418
6149,2022-12-06 15:54:00,15:54:00,2022-12-06,-0.010907,0.001016,0.007358,0.064478,-1.429822,-1.430803,-3.183557,-0.008065,-0.014232,-0.019973,0.051928,-1.879351,-1.988033,-3.866030,-0.001489,0.025202,0.080523,0.133291,-1.358125,-1.543554,-4.266790,0.000000,0.005490,0.030496,0.077090,-0.404652,-0.585979,-2.342200,0.000000,0.000000,0.000000,0.000000,-0.568436,-0.461389,-1.190838,0.012802,0.000000,-0.012546,0.064031,-0.445981,-0.051172,-1.226297,0.000000,0.000000,0.118982,0.118982,-1.450087,-1.55989,-3.363418
6150,2022-12-06 15:56:00,15:56:00,2022-12-06,-0.069262,-0.010907,-0.009891,0.073373,-1.440573,-1.441554,-3.194116,-0.080233,-0.008065,-0.022296,0.045234,-1.887264,-1.995938,-3.873783,0.000000,-0.001489,0.023713,0.096094,-1.359593,-1.545020,-4.268215,-0.040651,0.000000,0.005490,0.06101

In [110]:
df

,Datetime,time,date,Spx_ret,s_Spx_ret_1prd,s_Spx_ret_2prd,s_Spx_ret_4prd,s_Spx_ret_open,s_Spx_ret_close1,s_Spx_ret_close2,Nasdaq_ret,s_Nasdaq_ret_1prd,s_Nasdaq_ret_2prd,s_Nasdaq_ret_4prd,s_Nasdaq_ret_open,s_Nasdaq_ret_close1,s_Nasdaq_ret_close2,Russel_ret,s_Russel_ret_1prd,s_Russel_ret_2prd,s_Russel_ret_4prd,s_Russel_ret_open,s_Russel_ret_close1,s_Russel_ret_close2,EMXC_ret,s_EMXC_ret_1prd,s_EMXC_ret_2prd,s_EMXC_ret_4prd,s_EMXC_ret_open,s_EMXC_ret_close1,s_EMXC_ret_close2,EEMA_ret,s_EEMA_ret_1prd,s_EEMA_ret_2prd,s_EEMA_ret_4prd,s_EEMA_ret_open,s_EEMA_ret_close1,s_EEMA_ret_close2,EEM_ret,s_EEM_ret_1prd,s_EEM_ret_2prd,s_EEM_ret_4prd,s_EEM_ret_open,s_EEM_ret_close1,s_EEM_ret_close2,VTHR_ret,s_VTHR_ret_1prd,s_VTHR_ret_2prd,s_VTHR_ret_4prd,s_VTHR_ret_open,s_VTHR_ret_close1,s_VTHR_ret_close2
0,2022-10-21 09:30:00,09:30:00,2022-10-21,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2022-10-21 09:32:00,09:32:00,2022-10-21,0.011481,NaN,NaN,NaN,0.000000,NaN,NaN,0.095629,NaN,NaN,NaN,0.000000,NaN,NaN,-0.040538,NaN,NaN,NaN,0.000000,NaN,NaN,0.022023,NaN,NaN,NaN,0.000000,NaN,NaN,0.161958,NaN,NaN,NaN,0.000000,NaN,NaN,0.057746,NaN,NaN,NaN,0.000000,NaN,NaN,0.067049,NaN,NaN,NaN,0.000000,NaN,NaN
2,2022-10-21 09:34:00,09:34:00,2022-10-21,-0.203551,0.011481,NaN,NaN,0.011481,NaN,NaN,-0.369679,0.095629,NaN,NaN,0.095629,NaN,NaN,-0.258247,-0.040538,NaN,NaN,-0.040538,NaN,NaN,-0.110106,0.022023,NaN,NaN,0.022023,NaN,NaN,-0.071867,0.161958,NaN,NaN,0.161958,NaN,NaN,-0.129846,0.057746,NaN,NaN,0.057746,NaN,NaN,-0.091365,0.067049,NaN,NaN,0.067049,NaN,NaN
3,2022-10-21 09:36:00,09:36:00,2022-10-21,0.364942,-0.203551,-0.192094,NaN,-0.192094,NaN,NaN,0.371171,-0.369679,-0.274404,NaN,-0.274404,NaN,NaN,0.049602,-0.258247,-0.298680,NaN,-0.298680,NaN,NaN,0.305773,-0.110106,-0.088108,NaN,-0.088108,NaN,NaN,0.000000,-0.071867,0.089975,NaN,0.089975,NaN,NaN,0.288929,-0.129846,-0.072175,NaN,-0.072175,NaN,NaN,0.000000,-0.091365,-0.024377,NaN,-0.024377,NaN,NaN
4,2022-10-21 09:38:00,09:38:00,2022-10-21,-0.104203,0.364942,0.160648,NaN,0.172147,NaN,NaN,-0.028448,0.371171,0.000120,NaN,0.095749,NaN,NaN,-0.075841,0.049602,-0.208773,NaN,-0.249226,NaN,NaN,-0.077361,0.305773,0.195330,NaN,0.217396,NaN,NaN,0.000000,0.000000,-0.071867,NaN,0.089975,NaN,NaN,-0.057622,0.288929,0.158708,NaN,0.216546,NaN,NaN,-0.091458,0.000000,-0.091365,NaN,-0.024377,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6148,2022-12-06 15:52:00,15:52:00,2022-12-06,0.001016,0.006343,0.083273,0.108944,-1.430824,-1.431805,-3.184540,-0.014232,-0.005742,0.067544,0.132515,-1.865384,-1.974082,-3.852346,0.025202,0.055307,0.072364,0.155268,-1.382978,-1.568361,-4.290911,0.005490,0.025005,0.055525,0.116607,-0.410119,-0.591436,-2.347561,0.000000,0.000000,0.000000,0.030919,-0.568436,-0.461389,-1.190838,0.000000,-0.012546,0.064031,0.102491,-0.445981,-0.051172,-1.226297,0.000000,0.118982,0.118982,0.392025,-1.450087,-1.55989,-3.363418
6149,2022-12-06 15:54:00,15:54:00,2022-12-06,-0.010907,0.001016,0.007358,0.064478,-1.429822,-1.430803,-3.183557,-0.008065,-0.014232,-0.019973,0.051928,-1.879351,-1.988033,-3.866030,-0.001489,0.025202,0.080523,0.133291,-1.358125,-1.543554,-4.266790,0.000000,0.005490,0.030496,0.077090,-0.404652,-0.585979,-2.342200,0.000000,0.000000,0.000000,0.000000,-0.568436,-0.461389,-1.190838,0.012802,0.000000,-0.012546,0.064031,-0.445981,-0.051172,-1.226297,0.000000,0.000000,0.118982,0.118982,-1.450087,-1.55989,-3.363418
6150,2022-12-06 15:56:00,15:56:00,2022-12-06,-0.069262,-0.010907,-0.009891,0.073373,-1.440573,-1.441554,-3.194116,-0.080233,-0.008065,-0.022296,0.045234,-1.887264,-1.995938,-3.873783,0.000000,-0.001489,0.023713,0.096094,-1.359593,-1.545020,-4.268215,-0.040651,0.000000,0.005490,0.06101